## Analyzing the Image Data using the fastai library

In [ ]:
# import core modules
from fastai.vision import *
from fastai.metrics import accuracy
from fastai.metrics import r2_score
import numpy as np
from random import shuffle
import os

### Get the names of the task_folders

In [ ]:
# select the task name for the following analysis (only select one task name and then run all following 
# cells one after another)
taskname = "Point_Click_Task"
# taskname = "Drag_Drop_Task"
# taskname = "Follow_Circle_Task"
# taskname = "Slider_Task"

# get the current path
dir_path =  os.getcwd()

### Helper Function to do the Image Classification

In [ ]:
def get_databunch(directory, dv="condition"):
    
    # initialize variables to create the databunch correctly (depending on wether its a classification or
    # regression problem)
    if dv == "condition":
        # Helper Function to grab the dependent variable (label) from the image path name
        # name structure of file: taskname_participantid_condition_arousalValue_valenceValue.png
        func=lambda o: str(o)[-11]
        # label (None for classification, FloatList for regression)
        dv_label = None
    elif dv == "arousal":
        func=lambda o: str(o)[-9:-7]
        dv_label = FloatList
    elif dv == "valence":
        func=lambda o: str(o)[-6:-4]
        dv_label = FloatList
    elif dv == "task":
        func=lambda o: str(o)[-5]
        dv_label = None
    
    data = data = (ImageList.from_folder(directory)
                    .split_by_rand_pct()
                    .label_from_func(func, label_cls=dv_label)
                    .transform(size=224) # default 224
                    .databunch()
                    .normalize())
    
    return data


## Classification

### Classification without baseline (chose the images from the Condition Folder)

In [ ]:
print("Classification Without Baseline with task: ", taskname)
print("------------------------------------------------------")

# get the target path
targ_dir = os.path.join(dir_path, "Mouse_Usage_Images", "Cond", taskname)

# test = ImageList.from_folder(targ_dir).split_none()
# ll = test.label_from_func(func, label_cls=FloatList)

# create the databunch for the classification problem
databunch = get_databunch(targ_dir, "condition")

# print some examples of the images and their labels
databunch.show_batch(rows=3, figsize=(7,6))

In [ ]:
print("Classification Without Baseline with task: ", taskname)
print("------------------------------------------------------")

# create the learner
learn = cnn_learner(databunch, models.resnet34, metrics=accuracy)

# train the model
learn.fit_one_cycle(3)

In [ ]:
# get the predicted values and actual values
preds,targs = learn.get_preds()

# print some examples of the predicted and actual values to understand the output of the model
print(targs[:5])
# for classification, the class probability is detected --> get the index of the class with the highest probability
print([i.max(0)[1] for i in preds[:5]])

In [ ]:
print(len(preds))

### Classification with Baseline

In [ ]:
print("Classification With Baseline with task: ", taskname)
print("------------------------------------------------------")

# same procedure as with baseline, just uses the merged directory to get the merged image files
targ_dir = os.path.join(dir_path, "Mouse_Usage_Images", "Merged", taskname)

databunch = get_databunch(targ_dir, "condition")

# get examples
databunch.show_batch(rows=3, figsize=(7,6))

In [ ]:
print("Classification With Baseline with task: ", taskname)
print("------------------------------------------------------")

# create the learner
learn = cnn_learner(databunch, models.resnet34, metrics=accuracy)

# train the model
learn.fit_one_cycle(3)

In [ ]:
# get the predicted values and actual values and print examples
preds,targs = learn.get_preds()

print(targs[:5])
print([i.max(0)[1] for i in preds[:5]])
print(len(preds))

## Regression

### Regression without Baseline

#### Arousal

In [ ]:
print("Arousal Regression Without Baseline with task: ", taskname)
print("------------------------------------------------------")

# get the target path
targ_dir = os.path.join(dir_path, "Mouse_Usage_Images", "Cond", taskname)

# create the databunch for the regression problem and arousal as the dependent variable
databunch = get_databunch(targ_dir, "arousal")

# print some examples of the images and their labels
databunch.show_batch(rows=3, figsize=(7,6))

In [ ]:
print("Arousal Regression Without Baseline with task: ", taskname)
print("------------------------------------------------------")

# use the r² score for the regression
learn = cnn_learner(databunch, models.resnet34, metrics=r2_score)

learn.fit_one_cycle(3)

In [ ]:
# get the predicted values and actual values
preds,targs = learn.get_preds()

# print some examples of the predicted and actual values to understand the output of the model
print(targs[:5])
print(preds[:5])

#### Valence

In [ ]:
print("Valence Regression Without Baseline with task: ", taskname)
print("------------------------------------------------------")

targ_dir = os.path.join(dir_path, "Mouse_Usage_Images", "Cond", taskname)

# create the databunch with valence as the dependent variable
databunch = get_databunch(targ_dir, "valence")

databunch.show_batch(rows=3, figsize=(7,6))

In [ ]:
print("Valence Regression Without Baseline with task: ", taskname)
print("------------------------------------------------------")

learn = cnn_learner(databunch, models.resnet34, metrics=r2_score)

learn.fit_one_cycle(3)

In [ ]:
# get the predicted values and actual values
preds,targs = learn.get_preds()

print(targs[:5])
print(preds[:5])

### Regression With Baseline Data

#### Arousal

In [ ]:
print("Arousal Regression With Baseline with task: ", taskname)
print("------------------------------------------------------")

# get the target path (here: merged)
targ_dir = os.path.join(dir_path, "Mouse_Usage_Images", "Merged", taskname)

# create the databunch for the classification problem
databunch = get_databunch(targ_dir, "arousal")

# print some examples of the images and their labels
databunch.show_batch(rows=3, figsize=(7,6))

In [ ]:
print("Arousal Regression With Baseline with task: ", taskname)
print("------------------------------------------------------")

# create the learner
# use the r² score for the regression
learn = cnn_learner(databunch, models.resnet34, metrics=r2_score)

learn.fit_one_cycle(3)

In [ ]:
# get the predicted values and actual values
preds,targs = learn.get_preds()

print(targs[:5])
print(preds[:5])

#### Valence

In [ ]:
print("Valence Regression With Baseline with task: ", taskname)
print("------------------------------------------------------")

# get the target path (here: merged)
targ_dir = os.path.join(dir_path, "Mouse_Usage_Images", "Merged", taskname)

# create the databunch for the classification problem
databunch = get_databunch(targ_dir, "valence")

# print some examples of the images and their labels
databunch.show_batch(rows=3, figsize=(7,6))

In [ ]:
print("Valence Regression With Baseline with task: ", taskname)
print("------------------------------------------------------")

# create the learner
# use the r² score for the regression
learn = cnn_learner(databunch, models.resnet34, metrics=r2_score)

learn.fit_one_cycle(3)

In [ ]:
# get the predicted values and actual values
preds,targs = learn.get_preds()

print(targs[:5])
print(preds[:5])

### Task versus Task Classification (Proof-of-Principle)

In [ ]:
print("Task Versus Task Classification: Point Click vs Drag Drop")
print("------------------------------------------------------")

# get the target path (here: merged)
targ_dir = os.path.join(dir_path, "Mouse_Usage_Images", "TaskVSTask")

# create the databunch for the task classification problem
databunch = get_databunch(targ_dir, dv="task")

# print some examples of the images and their labels
databunch.show_batch(rows=3, figsize=(7,6))

In [ ]:
print("Task Versus Task Classification: Point Click vs Drag Drop")
print("------------------------------------------------------")

# create the learner
# use the r² score for the regression
learn = cnn_learner(databunch, models.resnet34, metrics=accuracy)

learn.fit_one_cycle(3)

In [32]:
# get the predicted values and actual values
preds,targs = learn.get_preds()

In [ ]:
# get the number of images that were predicted
len(targs)

In [ ]:
# print some examples of the predicted and actual values to understand the output of the model
print(targs[:5])
# for classification, the class probability is detected --> get the index of the class with the highest probability
print([i.max(0)[1] for i in preds[:5]])